In [20]:
from transformers import pipeline
import torch
import pandas as pd
import re
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import folium

In [9]:
pipe = pipeline("token-classification", model="akdeniz27/bert-base-turkish-cased-ner")

c:\Users\Enes\anaconda3\envs\gputorch\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


# Example

In [10]:
text = "kahramanmaraş elbistan pınarbaşı mahallesi pınarbaşı caddesi"

result = pipe(text)

print("Token ve Etiketler:")
location = ""  
for entity in result:
    token = entity['word']
    label = entity['entity']
    
    print(f"Token: {token}, Etiket: {label}")
    if 'LOC' in label:  
        if not token.startswith('##'):
                if location:  
                    location += " "  
                location += token
        else:
                location += token.lstrip('##')  


print("\nLocation:")
print(location)


Token ve Etiketler:
Token: kahraman, Etiket: B-LOC
Token: ##maraş, Etiket: B-LOC
Token: elbis, Etiket: B-LOC
Token: ##tan, Etiket: B-LOC
Token: p, Etiket: B-LOC
Token: ##ınar, Etiket: B-LOC
Token: ##başı, Etiket: B-LOC
Token: p, Etiket: B-LOC
Token: ##ınar, Etiket: B-LOC
Token: ##başı, Etiket: B-LOC

Location:
kahramanmaraş elbistan pınarbaşı pınarbaşı


In [11]:
text = "kahramanmaraş elbistan pınarbaşı mahallesi pınarbaşı caddesi"

result = pipe(text)

print("Token ve Etiketler:")

location = ""
address_keywords = ["mahallesi", "caddesi", "sokak", "bulvarı", "köyü", "yolu", "mevkii"]

loc_tokens = []
for entity in result:
    token = entity['word']
    label = entity['entity']
    print(f"Token: {token}, Etiket: {label}")

    if "LOC" in label:  
        if token.startswith("##"):
            loc_tokens[-1] += token.lstrip("##")  
        else:
            loc_tokens.append(token)

words = text.split()  
for i, word in enumerate(words):
    if any(loc in word for loc in loc_tokens):
        location += word + " " 
    if any(keyword in word.lower() for keyword in address_keywords):
            location += word + " " 

print("\nLocation:")
print(location.strip())

Token ve Etiketler:
Token: kahraman, Etiket: B-LOC
Token: ##maraş, Etiket: B-LOC
Token: elbis, Etiket: B-LOC
Token: ##tan, Etiket: B-LOC
Token: p, Etiket: B-LOC
Token: ##ınar, Etiket: B-LOC
Token: ##başı, Etiket: B-LOC
Token: p, Etiket: B-LOC
Token: ##ınar, Etiket: B-LOC
Token: ##başı, Etiket: B-LOC

Location:
kahramanmaraş elbistan pınarbaşı mahallesi pınarbaşı caddesi


In [12]:
df = pd.read_csv('tweets_cleaned.csv')
df

,timestamp,username,handle,tweet_date,content,content_ment_link,content_wo_punct,content_wo_removed_english,content_wo_normalize,content_wo_tokenize,content_wo_stop,content_wo_lemmatized,content_no_rare_words
0,2024-11-03 19:38:25,Orhan Şevik,orhan_sevik,"Feb 8, 2023",@haluklevent\n @oguzhanugur\n hepinizden alla...,\n \n hepinizden allah razı olsun iyiki varsı...,hepinizden allah razı olsun iyiki varsını...,hepinizden allah razı olsun iyiki varsınız deprem,hepinizden allah razı olsun iyi ki varsınız de...,"['hepinizden', 'allah', 'razı', 'olsun', 'iyi'...","['hepinizden', 'razı', 'varsınız', 'deprem']","['hep', 'razı', 'var', 'deprem']","['hep', 'razı', 'var', 'deprem']"
1,2024-11-03 19:38:25,nursena,nurssxx_,"Feb 8, 2023",allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver deprem,allahım nolur onlara dayanma gücü ver deprem,allahım ne olur onlara dayanma gücü ver deprem,"['allahım', 'ne', 'olur', 'onlara', 'dayanma',...","['onlara', 'dayanma', 'gücü', 'ver', 'deprem']","['o', 'dayan', 'güc', 'ver', 'deprem']","['o', 'dayan', 'güc', 'ver', 'deprem']"
2,2024-11-03 19:38:25,Elif ŞEKER,55ellllllif,"Feb 8, 2023",replying to \n@haluklevent\n @danlabilic\n and...,replying to \n\n \n and \n_harun\nadıyamanda a...,replying to and harun adıyamanda ali taş...,harun adıyamanda ali taşı mahallesi sokak hic...,harun adıyamanda ali taşı mahallesi sokak hicr...,"['harun', 'adıyamanda', 'ali', 'taşı', 'mahall...","['harun', 'adıyamanda', 'ali', 'taşı', 'mahall...","['haru', 'adıyaman', 'ali', 'taşı', 'mahalle',...","['adıyaman', 'ali', 'taşı', 'mahalle', 'sokak'..."
3,2024-11-03 19:38:25,Yusuf,yusufaltuns,"Feb 8, 2023",tuğba södekoğlu kovulsun \n@showtv\n #deprem,tuğba södekoğlu kovulsun \n\n #deprem,tuğba södekoğlu kovulsun deprem,tuğba södekoğlu kovulsun deprem,tuğba sodekoğlu kovulsun deprem,"['tuğba', 'sodekoğlu', 'kovulsun', 'deprem']","['tuğba', 'sodekoğlu', 'kovulsun', 'deprem']","['tuğba', 'sodekoğlu', 'kov', 'deprem']",['deprem']
4,2024-11-03 19:38:25,𝐎̈𝐳𝐠𝐮̈𝐫 𝐑𝐚𝐧,dryghtn2,"Feb 8, 2023",arkadaşimiza ulaşamiyoruz\nkahramanmaraş elbis...,arkadaşimiza ulaşamiyoruz\nkahramanmaraş elbis...,arkadaşimiza ulaşamiyoruz kahramanmaraş elbist...,arkadaşimiza ulaşamiyoruz kahramanmaraş elbist...,arkadaşımıza ulaşamıyoruz kahramanmaraş elbist...,"['arkadaşımıza', 'ulaşamıyoruz', 'kahramanmara...","['arkadaşımıza', 'ulaşamıyoruz', 'kahramanmara...","['arkadaş', 'ulaş', 'kahramanmaraş', 'elbistan...","['arkadaş', 'ulaş', 'kahramanmaraş', 'elbistan..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2097,2024-11-17 16:28:31,KEREMCGRC,keremcgrci,"Feb 8, 2023",yardım ağı´na destek ol! \n\ntwitter'da paylaş...,yardım ağı´na destek ol! \n\ntwitter'da paylaş...,yardım ağı´na destek ol twitterda paylaşılan...,yardım ağı´na destek ol twitterda paylaşılan a...,yardım ağına destek ol twitterda paylaşılan ad...,"['yardım', 'ağına', 'destek', 'ol', 'twitterda...","['yardım', 'ağına', 'destek', 'ol', 'twitterda...","['yar', 'ağın', 'destek', 'ol', 'twitter', 'pa...","['yardım', 'destek', 'ol', 'paylaş', 'adres', ..."
2098,2024-11-17 16:28:31,aMMo,tatavaypmadvmet,"Feb 8, 2023",teyi̇tli̇ lütfen yardim\nemlak bank evleri 1. ...,teyi̇tli̇ lütfen yardim\nemlak bank evleri 1. ...,teyi̇tli̇ lütfen yardim emlak bank evleri kıs...,teyi̇tli̇ lütfen yardim emlak evleri kısım blo...,teyi̇tli̇ lütfen yardım emlak evleri kısım blo...,"['teyi̇tli̇', 'lütfen', 'yardım', 'emlak', 'ev...","['teyi̇tli̇', 'yardım', 'emlak', 'evleri', 'kı...","['teyi̇tli̇', 'yar', 'emlak', 'ev', 'kısım', '...","['teyi̇tli̇', 'yardım', 'ev', 'blok', 'hatay',..."
2099,2024-11-17 16:28:31,N’Y,nyonyedi,"Feb 8, 2023",05324707903 babalatv\n\nwhastapp hatti tüm böl...,05324707903 babalatv\n\nwhastapp hatti tüm böl...,babalatv whastapp hatti tüm bölgelerdeki yar...,babalatv whastapp hatti tüm bölgelerdeki yardi...,babalatv Whatsapp hattı tüm bölgelerdeki yardı...,"['babalatv', 'Whatsapp', 'hattı', 'tüm', 'bölg.

# Get Location 

In [13]:
address_keywords = ["mahallesi", "caddesi", "sokak", "bulvarı", "köyü", "yolu", "mevkii", "mah.", "cad.", "sok.", "bul.", "köy.", "yol.", "mek.","apartman","apt"]

In [14]:
def get_location_from_text(text):
    if not text.strip():
        return None 
    result = pipe(text) 
    location = "" 
    
    for entity in result:
        token = entity['word']
        label = entity['entity']

        if 'LOC' in label:  
            if not token.startswith('##'):  
                if location:  
                    location += " " 
                location += token
            else:
                location += token.lstrip('##')  
    words = text.split()  
    for i, word in enumerate(words):
        if any(loc in word for loc in loc_tokens):
            location += word + " " 
        if any(keyword in word.lower() for keyword in address_keywords):
            location += word + " " 

    return location.strip() if location else None
df['location'] = df['content_wo_normalize'].apply(get_location_from_text)

# Get latitude & longitude

### Example

In [15]:
geolocator = Nominatim(user_agent="geoapi")

# Adres olarak tam metni belirtiyoruz
address = "hatay antakya cebrail mahallesi"

# Adresi enlem ve boylama çevir
location = geolocator.geocode(address)

if location:
    print(f"Adres: {address}")
    print(f"Enlem: {location.latitude}, Boylam: {location.longitude}")
else:
    print(f"Adres bulunamadı: {address}")

Adres: hatay antakya cebrail mahallesi
Enlem: 36.2063404, Boylam: 36.1565635


## Get Coordinates

In [17]:
geolocator = Nominatim(user_agent="geoapi")

def get_coordinates(address):
    if not address or pd.isna(address):
        return None, None
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        return None, None


In [18]:
df[['latitude', 'longitude']] = df['location'].apply(
    lambda x: pd.Series(get_coordinates(x))
)

In [19]:
df.to_csv('tweets_with_location.csv', index=False)

## Draw Map

In [21]:
df_valid = df.dropna(subset=['latitude', 'longitude'])

m = folium.Map(location=[41.0369, 28.9850], zoom_start=6)

for _, row in df_valid.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['location'] 
    ).add_to(m)

m.save("harita.html") 